In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# from sklearn.datasets import load_boston

In [2]:
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]

In [3]:
features=np.array(data)
labels=np.array(target)

In [4]:
print(features)
print(labels)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.6 19.3 20.1 19.5 19.5 20.4 19.8 19.

In [5]:
features.shape

(506, 13)

In [6]:
n_training_samples=features.shape[0]
print(n_training_samples)
n_dim=features.shape[1]
print(n_dim)

506
13


In [7]:
def normalise(dataset):
    mu=np.mean(dataset,axis=0)
    sigma=np.std(dataset,axis=0)
    return (dataset-mu)/sigma
features_norm=normalise(features)

In [8]:
features_norm

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [9]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(features_norm,labels,test_size=0.33,random_state=42)
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(339, 13)
(167, 13)
(339,)
(167,)


In [10]:
xtrain=np.transpose(xtrain)
xtest=np.transpose(xtest)
ytrain=np.transpose(ytrain)
ytest=np.transpose(ytest)

In [11]:
xtrain.shape

(13, 339)

In [12]:
ytrain=ytrain.reshape(1,len(ytrain))
print(ytrain.shape)

(1, 339)


In [13]:
## construction state

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
tf.reset_default_graph()

In [16]:
n_dim

13

In [17]:
x=tf.placeholder(tf.float32,[n_dim,None])
y=tf.placeholder(tf.float32,[1,None])
learning_rate=tf.placeholder(tf.float32,shape=())
w=tf.Variable(tf.ones([n_dim,1]))
b=tf.Variable(tf.zeros(1))
init=tf.global_variables_initializer()
y_=tf.matmul(tf.transpose(w),x)+b
cost=tf.reduce_mean(tf.square(y_-y))
training_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

## evaluation state

In [18]:
def run_linear_model(learning_r,training_epochs,train_obs,train_label,debug=False):
    sess=tf.Session()
    sess.run(init)
    cost_history=np.empty(shape=[0],dtype=float)
    for epoch in range(training_epochs):
        sess.run(training_step,feed_dict={x:train_obs,y:train_label,learning_rate:learning_r})
        cost_=sess.run(cost,feed_dict={x:train_obs,y:train_label})
        if (epoch%10==1) & debug:
            print('reached epoch',epoch,'cost j =',cost_)
    return sess,cost_history

In [19]:
sess,cost_history=run_linear_model(learning_r=0.01,training_epochs=2000,train_obs=xtrain,train_label=ytrain,debug=True)

reached epoch 1 cost j = 586.51025
reached epoch 11 cost j = 360.71655
reached epoch 21 cost j = 243.49263
reached epoch 31 cost j = 169.55118
reached epoch 41 cost j = 121.34518
reached epoch 51 cost j = 89.60208
reached epoch 61 cost j = 68.55571
reached epoch 71 cost j = 54.51298
reached epoch 81 cost j = 45.08405
reached epoch 91 cost j = 38.71153
reached epoch 101 cost j = 34.37405
reached epoch 111 cost j = 31.398296
reached epoch 121 cost j = 29.338121
reached epoch 131 cost j = 27.896694
reached epoch 141 cost j = 26.87566
reached epoch 151 cost j = 26.141962
reached epoch 161 cost j = 25.605972
reached epoch 171 cost j = 25.2071
reached epoch 181 cost j = 24.904196
reached epoch 191 cost j = 24.66919
reached epoch 201 cost j = 24.482819
reached epoch 211 cost j = 24.331793
reached epoch 221 cost j = 24.20687
reached epoch 231 cost j = 24.101583
reached epoch 241 cost j = 24.011343
reached epoch 251 cost j = 23.932875
reached epoch 261 cost j = 23.863792
reached epoch 271 cost 

In [20]:
loss=tf.keras.losses.MSE(ytest,y_)
print(sess.run(loss,feed_dict={x:xtest}))
sess.close

[20.80238]


<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x0000020332EC85D0>>